In [1]:
import qiniu

初始化，获取签名授权

In [2]:
access_key = "x34GCNDuYpCFsNtqkrri3Eagr6OANYyboWRZtWQv"
secret_key = "f_lfzdkBoEfOa_7a_0yrMOcIdfr-cHv9H463B5Xh"
# access_key = "MUpLpPtmVeiP7UWWteuSRBm5cHNeqSMndp4zh339"
# secret_key = "0ix4DVQZvPdlQNDIn0qTudaBj8h27FK9qWThMd5L"

# 构建鉴权对象
q = qiniu.Auth(access_key, secret_key)

数据或文件上传

In [11]:
# data upload
# key = 'pupy dog here.mp4'
# data = 'hello qiniu!'
# token = q.upload_token("yy-short-video")
# ret, info = qiniu.put_data(token, key, data)

# file upload
token = q.upload_token("yy-short-video")
key = 'pupy dog here.mp4'
file_path = './1185308075.mp4'
ret, info = qiniu.put_file(token, key, file_path, metadata={"name": "xx"})
if ret is not None:
    print('All is OK', ret)
else:
    print(info)

All is OK {'hash': 'FkwkxLPsVhf1C3FK7EsSVprQLktO', 'key': 'pupy dog here.mp4'}


资源管理

In [3]:
# 初始化 BucketManager 空间管理类
bucket = qiniu.BucketManager(q)

# 指定测试空间与文件名
bucket_name = "yy-short-video"
key = "pupy dog here.mp4"

ret, info = bucket.stat(bucket_name, key)

In [9]:
prefix = None
# 列举条目
limit = 1
# 列举出除'/'的所有文件以及以'/'为分隔的所有前缀
delimiter = None
# 标记
marker = None
ret, eof, info = bucket.list(bucket_name, prefix, marker, limit, delimiter)
print(info)

_ResponseInfo__response:<Response [200]>, exception:None, status_code:200, text_body:{"marker":"eyJjIjowLCJrIjoicHVweSBkb2cgaGVyZS5tcDQifQ==","items":[{"key":"pupy dog here.mp4","hash":"FkwkxLPsVhf1C3FK7EsSVprQLktO","fsize":1929256,"mimeType":"video/mp4","putTime":16983134286283495,"type":0,"status":0,"md5":"1954158dfcc00fa37bca49bbda9cfc2a"}]}, req_id:MbYAAADfqIYgspEX, MbYAAADfqIYgspEX.peer, x_log:redis.g;redis.g;redis.g;RsDbClusterID:id_hash;v4mget;dirCount:0 itemCount:1 fdelCount:0;rs5_shard_hash.ls:3;RSF:4;RSF:7


In [8]:
info.x_log

'redis.g;redis.g;redis.g;RsDbClusterID:id_hash;v4mget;dirCount:0 itemCount:2 fdelCount:0;rs5_shard_hash.ls:3;RSF:4;RSF:7'

资源处理

In [12]:
# 是使用的队列名称,不设置代表不使用私有队列，使用公有队列。
pipeline = "small_eye"

# 要进行转码的转码操作。
fops = "vframe/jpg/offset/7/w/480/h/360"

saveas_key = qiniu.urlsafe_base64_encode(f"{bucket_name}:{key.split('.')[0]}_frame7.jpg")
fops = fops+"|saveas/"+saveas_key

pfop = qiniu.PersistentFop(q, bucket_name, pipeline)
ops = [fops]
ret, info = pfop.execute(key, ops, 1)

In [13]:
ret

{'persistentId': 'cn-east-2.01c201cwi9wdvuim3c00mvdajq000akz'}